<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/3.1-Deezy-Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install DeezyMatch
!pip install mmh3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 

In [8]:
import os
import tqdm
import time
import sys
import json
import pandas as pd
import tracemalloc
from DeezyMatch import train as dm_train, plot_log, finetune as dm_finetune, inference as dm_inference, combine_vecs, candidate_ranker, candidate_ranker_init


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject


In [3]:
!git clone https://github.com/stavco9/datastreaming-final-project.git

fatal: destination path 'datastreaming-final-project' already exists and is not an empty directory.


In [6]:
sys.path.insert(1, './datastreaming-final-project/modules')
from CountMinSketch.count_min_sketch2 import CountMinSketch2
from BloomFilter.bloom_filter2 import BloomFilter2

In [9]:
words_set = []
for i in range(10):
  candidates_file = f"./Dataset/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = []
    for article in list(json.loads(f.read()).values()):
      words_list.extend(article)
    words_set.append({"words_list": words_list})

In [10]:
NUM_COUNTERS = 300000
NUM_HASH_FUNCTIONS = 100

for index, words_list in enumerate(words_set):
    print(f"Init CMS - {index + 1} of {len(words_set)}")
    cms = CountMinSketch2(buckets=NUM_COUNTERS, tables=NUM_HASH_FUNCTIONS)
    for word in words_list["words_list"]:
        cms.increment(word)
    words_set[index]['words_cms'] = cms
    del cms

Init CMS - 1 of 10
Init CMS - 2 of 10
Init CMS - 3 of 10
Init CMS - 4 of 10
Init CMS - 5 of 10
Init CMS - 6 of 10
Init CMS - 7 of 10
Init CMS - 8 of 10
Init CMS - 9 of 10
Init CMS - 10 of 10


In [11]:
p = 0.01 #false positive probability

for index, words_list in enumerate(words_set):
    print(f"Init BLOOM - {index + 1} of {len(words_set)}")
    n = len(words_list["words_list"]) #no of items to add
    bloomf = BloomFilter2(n,p)
    for word in words_list["words_list"]:
        bloomf.add(word)
    words_set[index]['words_bloomf'] = bloomf
    del bloomf

Init BLOOM - 1 of 10
Init BLOOM - 2 of 10
Init BLOOM - 3 of 10
Init BLOOM - 4 of 10
Init BLOOM - 5 of 10
Init BLOOM - 6 of 10
Init BLOOM - 7 of 10
Init BLOOM - 8 of 10
Init BLOOM - 9 of 10
Init BLOOM - 10 of 10


In [ ]:
for i in range(10):
  print(f"Dataset {i + 1} of 10")
  # generate vectors for candidates (specified in dataset_path)
  # using a model stored at pretrained_model_path and pretrained_vocab_path
  dm_inference(input_file_path="./DeezyMatchDataset/train_conf.yaml",
              dataset_path=f"./Dataset/dataset_candidates_{i}.txt",
              pretrained_model_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.model",
              pretrained_vocab_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.vocab",
              inference_mode="vect",
              scenario=f"./DeezyMatchDataset/candidates/test_{i}")
  print("========================")

In [ ]:
for i in range(10):
  print(f"Dataset {i + 1} of 10")
  # combine vectors stored in candidates/test and save them in combined/candidates_test
  combine_vecs(rnn_passes=['fwd', 'bwd'],
              input_scenario=f'./DeezyMatchDataset/candidates/test_{i}',
              output_scenario=f'./DeezyMatchDataset/combined/candidates_test_{i}',
              print_every=10)
  print("========================")

Streaming output truncated to the last 5000 lines.
0000330 candidates/test_2/embeddings/rnn_fwd_330
0000340 candidates/test_2/embeddings/rnn_fwd_340
0000350 candidates/test_2/embeddings/rnn_fwd_350
0000360 candidates/test_2/embeddings/rnn_fwd_360
0000370 candidates/test_2/embeddings/rnn_fwd_370
0000380 candidates/test_2/embeddings/rnn_fwd_380
0000390 candidates/test_2/embeddings/rnn_fwd_390
0000400 candidates/test_2/embeddings/rnn_fwd_400
0000410 candidates/test_2/embeddings/rnn_fwd_410
0000420 candidates/test_2/embeddings/rnn_fwd_420
0000430 candidates/test_2/embeddings/rnn_fwd_430
0000440 candidates/test_2/embeddings/rnn_fwd_440
0000450 candidates/test_2/embeddings/rnn_fwd_450
0000460 candidates/test_2/embeddings/rnn_fwd_460
0000470 candidates/test_2/embeddings/rnn_fwd_470
0000480 candidates/test_2/embeddings/rnn_fwd_480
0000490 candidates/test_2/embeddings/rnn_fwd_490
0000500 candidates/test_2/embeddings/rnn_fwd_500
0000510 candidates/test_2/embeddings/rnn_fwd_510
0000520 candidates

In [12]:
def format_word(word):
  if len(word) > 1:
    last_char = word[-1]
    while not last_char.isalpha():
      word = word[:-1]
      last_char = word[-1]

    first_char = word[0]
    while not first_char.isalpha():
      word = word[1:]
      first_char = word[0]

    word = word.lower()
  elif word.isalpha():
    word = word.lower()

  return word

In [13]:
#some_word_with_error = "appel"
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
  or a wartermelon if possibe. I hope i don't have an errof in Londaon and in Jerusealm"
correct_sentence = "I wanna apple please or a banana with a pear and a lemon, \
  or a watermelon if possible. I hope i don't have an error in London and in Jerusalem"
diff_words_to_correct = list(set([format_word(x) for x in correct_sentence.split()]) - set([format_word(x) for x in some_sentance_with_error.split()]))
new_sentance = []
words_set_out = []

In [15]:
for index in range(10):
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  tracemalloc.start()
  time_start_deezy = time.time()

  ranker = candidate_ranker_init(candidate_scenario=f"./DeezyMatchDataset/combined/candidates_test_{index}",
                      query=[format_word(x) for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=2,
                      search_size=100,
                      output_path=f"./DeezyMatchDataset/ranker_results/test_candidates_deezymatch_on_the_fly_{index}",
                      pretrained_model_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./DeezyMatchDataset/models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))
  ranker.rank()

  time_end_deezy = time.time()
  words_set[index]["total_deezy_mem"] = tracemalloc.get_traced_memory()[0]
  tracemalloc.stop()
  words_set[index]["deezy_match_df"] = ranker.output
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-31 17:07:26 88c41c55be7e [INFO] read input file: ./DeezyMatchDataset/combined/candidates_test_0/train_conf.yaml
2024-03-31 17:07:26 88c41c55be7e [INFO] pytorch will use: cuda
2024-03-31 17:07:27 88c41c55be7e [INFO] use a dataframe in test_tokenize.
2024-03-31 17:07:27 88c41c55be7e [INFO] number of labels, True: 0 and False: 30
2024-03-31 17:07:27 88c41c55be7e [INFO] skipping 0 lines


2024-03-31 17:07:27 88c41c55be7e [INFO] save test-data-class: tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_fwd*
0000000 tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_bwd*
0000000 tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_65edb892-8c4c-4936-b223-a25d7cbec9f7/query/embeddings/rnn_indxs_0
--- 716.536740064621 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 i
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 5, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 30, searched: 100
=========== Start the search for 3 pleases
ID: 4/30 

2024-03-31 17:07:30 88c41c55be7e [INFO] save test-data-class: tmp_fdc7430d-2caf-4c93-bc6e-af219f31c8e0/query/dataframe.df


...

  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_fwd*
0000000 tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_bwd*
0000000 tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_d09dad1e-fae8-4441-9711-7316dc53a7d4/query/embeddings/rnn_indxs_0
--- 737.7193946838379 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 i
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 29, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

2024-03-31 17:07:51 88c41c55be7e [INFO] save test-data-class: tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_fwd*
0000000 tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_bwd*
0000000 tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_f6076449-3a07-465b-a14d-10ab76c10a8d/query/embeddings/rnn_indxs_0
--- 740.3612835407257 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 i
ID: 1/30 -- Number of found candidates so far: 2, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 33, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

In [16]:
words_set_out = []
for index, words_list in enumerate(words_set):
    tracemalloc.start()
    start = time.time()
    deezy_match_df = words_set[index]["deezy_match_df"]
    words_set_out.append({})
    words_set_out[index]["matched_words"] = []
    words_set_out[index]["match_distance"] = []
    words_set_out[index]["words_frequency"] = []
    start = time.time()
    for word in some_sentance_with_error.split():
        word = format_word(word)
        if not word in words_list['words_bloomf']:
            print(f"Scanning {word} with Deezy Match")
            print("==========================")
            word_df_row = deezy_match_df.loc[deezy_match_df['query'] == word].iloc[0]
            match_word = set([format_word(x) for x in list(set(word_df_row['faiss_distance'].keys()))]) if bool(word_df_row['faiss_distance']) else None
            match_distance = set(word_df_row['faiss_distance'].values()) if bool(word_df_row['faiss_distance']) else -1
            list_freq = [words_list["words_cms"].count(x[0]) for x in list(match_word)] if bool(word_df_row['faiss_distance']) else -1
            words_set_out[index]["matched_words"].append(match_word)
            words_set_out[index]["match_distance"].append(match_distance)
            words_set_out[index]["words_frequency"].append(list_freq)
            for idx, fix in enumerate(list(set(match_word))):
              print(f"Potential fix of {word} is {fix}")
              print(f"Frequency is {list(list_freq)[idx] if idx < len(list_freq) else list(list_freq)[-1]}")
              print(f"Match distance is {list(match_distance)[idx] if idx < len(match_distance) else list(match_distance)[-1]}")
              print("==========================")
        else:
            print(f"Word {word} is correct. Skipping")
            words_set_out[index]["matched_words"].append(None)
            words_set_out[index]["match_distance"].append(None)
            words_set_out[index]["words_frequency"].append(None)

    end = time.time()
    words_set_out[index]["total_memory_mb"] = (words_set[index]["total_deezy_mem"] + tracemalloc.get_traced_memory()[0]) / 1024 / 1024
    tracemalloc.stop()
    print(f"Total runtime of {(end - start)} seconds")
    words_set_out[index]["time_sec"] = words_list["time_deezy_sec"] + (end - start)

Word i is correct. Skipping
Scanning wanna with Deezy Match
Potential fix of wanna is wanna
Frequency is 2.0
Match distance is 0.0
Scanning appel with Deezy Match
Potential fix of appel is appealed
Frequency is 5143.0
Match distance is 4.7146
Potential fix of appel is papel
Frequency is 5.0
Match distance is 6.1974
Scanning pleases with Deezy Match
Potential fix of pleases is settles
Frequency is 18.0
Match distance is 5.075
Potential fix of pleases is spieletester.com
Frequency is 18.0
Match distance is 5.0393
Word or is correct. Skipping
Word a is correct. Skipping
Scanning bannana with Deezy Match
Potential fix of bannana is bandanna
Frequency is 952.0
Match distance is 3.2076
Potential fix of bannana is nabāto
Frequency is 10.0
Match distance is 6.1802
Word with is correct. Skipping
Word a is correct. Skipping
Scanning paer with Deezy Match
Potential fix of paer is parameter
Frequency is 5.0
Match distance is 3.4475
Potential fix of paer is parker
Frequency is 5.0
Match distance is

In [17]:
OUT_DIR = './output/3-deezy-alg'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

In [ ]:
print(words_set_out)

[{'matched_words': [None, None, {'papel', 'appealed'}, {'settles', 'spieletester.com'}, None, None, {'nabāto', 'bandanna'}, None, None, {'parker', 'parameter'}, None, None, {'lemon', 'leonie'}, None, None, {'warned', 'trawled'}, None, {'ps', 'pose'}, None, None, None, None, None, None, {'error', 'r'}, None, {'london', 'longwy'}, None, None, {'jerusalem', 'describes'}], 'match_distance': [None, None, {4.7146, 6.1974}, {5.075, 5.0393}, None, None, {3.2076, 6.1802}, None, None, {3.4475, 4.5089}, None, None, {5.673, 6.2809}, None, None, {7.5992, 7.4863}, None, {7.2907, 7.2001}, None, None, None, None, None, None, {4.2706, 6.4512}, None, {3.8871, 5.7876}, None, None, {5.5064, 6.7379}], 'words_frequency': [None, None, [9.0, 4922.0], [35.0, 35.0], None, None, [18.0, 759.0], None, None, [9.0, 9.0], None, None, [17.0, 17.0], None, None, [4.0, 103.0], None, [9.0, 9.0], None, None, None, None, None, None, [189.0, 8.0], None, [17.0, 17.0], None, None, [23.0, 132.0]], 'total_memory_mb': 3.620795249

In [18]:
for index, words_list in enumerate(words_set_out):
    df_comparisons = pd.DataFrame(data=words_list, index=[format_word(x) for x in correct_sentence.split()])
    df_comparisons = df_comparisons.groupby(df_comparisons.index).first()
    df_comparisons.to_csv(f'{OUT_DIR}/dataset_{index}.csv')

In [19]:
df_comparisons

,matched_words,match_distance,words_frequency,total_memory_mb,time_sec
a,None,None,None,3.558549,2.743487
an,None,None,None,3.558549,2.743487
and,None,None,None,3.558549,2.743487
apple,"{appealed, padel}","{4.8308, 6.1974}","[5077.0, 12.0]",3.558549,2.743487
banana,"{bandanna, nana}","{3.2076, 5.6122}","[903.0, 16.0]",3.558549,2.743487
don't,None,None,None,3.558549,2.743487
error,"{fervor, error}","{4.2706, 5.9552}","[76.0, 205.0]",3.558549,2.743487
have,None,None,None,3.558549,2.743487
hope,None,None,None,3.558549,2.743487
i,None,None,None,3.558549,2.743487
